# GridION COVID RUNS

${SNPDB[$item]}----------

## GRAB DATA
start or resume a screen session
```bash
 screen -r sync # screen -S sync
```

find RUN_ID:

- W:\COVID-19\sequencing\Runs
  - example: W:\COVID-19\sequencing\Runs\Artic_V3\20210106

export it:
```bash
 export RUN_ID=20210203
```

sync it (here is an example using KSC GridION, for other sites, please make sure  to change to the correct IP addresses and directories, see next section "SSH Login IP address and run directories":
```bash
rsync -zavm --include="*/" --exclude '*.fast5' --exclude '*/fast5_fail/*' --exclude '*/fastq_fail/*' --perms --group --chmod=D0770,F0640 --groupmap=*:SEC_lab_VIR grid@10.1.30.16:/data/${RUN_ID} /NGS/active/VIR/SARS-CoV2/run_links/GridIon
```

give gridion password

detach screen (CRTL+d)

wait / monitor progress



## SSH Login IP address and run directories

_KSC GridIon_
- connection:                 ssh grid@10.1.30.16
- path on Gridon:             /data/${RUN_ID} 
- path on production server: /NGS/active/VIR/SARS-CoV2/run_links/GridIon/

_MASC GridIon_  
- connection: ssh grid@10.2.31.23
- path on Gridon: /data/COVID19/RAPID/${RUN_ID} #two folders for different protocols:  _ARTIC|RAPID_
- path on production server: /NGS/active/VIR/SARS-CoV2/run_links/MASC/

_CSC GridIon_: 
- connection: ssh grid@10.3.30.100  
- path on Gridon: /data/GridION_CSC/COVID19/RAPID/${RUN_ID} #two folders for different protocols:  _ARTIC|RAPID_
- path on production server: /NGS/active/VIR/SARS-CoV2/run_links/CSC/


-------

## Assembly
start or resume a screen session
```bash
screen -r assem # screen -S assem
```

export RUN_ID
```bash
 export RUN_ID=20210106
```

navigate to the SARS directory
```bash
cd /NGS/active/VIR/SARS-CoV2
```

### need to know both the amplicon design and location of the run!  
location of the run is defined by:  
-l M or C or K: M=MASC; C=CSC, K=KSC  

### Rapid Assembly script has been consilidated!

**NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_rapid.sh -c ./Artic_pipeline_config -l "M, C or K" -p V1200 $RUN_ID**


### Rapid run script has been consilidated please see above

## Run_scripts:

### MASC gridIon artic run

    rsync -zavmr --include="*/" --exclude="*.fast5" --exclude="*/fastq_fail/*" --chmod=D0770,F0640 --groupmap=*:SEC_lab_VIR   grid@10.2.31.23:/data/COVID19/ARTIC/${RUN_ID} /NGS/active/VIR/SARS-CoV2/run_links/MASC/

    bash NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_artic_96_run.sh -c ./Artic_pipeline_config -l M $RUN_ID

### MASC gridIon rapid run
    rsync -zavmr --include="*/" --exclude="*.fast5" --exclude="*/fastq_fail/*" --chmod=D0770,F0640 --groupmap=*:SEC_lab_VIR   grid@10.2.31.23:/data/COVID19/RAPID/${RUN_ID} /NGS/active/VIR/SARS-CoV2/run_links/MASC/
    
    NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_rapid.sh -c ./Artic_pipeline_config -l M -p V1200 $RUN_ID
  
### MASC gridIon artic V4 run
    rsync -zavmr --include="*/" --exclude="*.fast5" --exclude="*/fastq_fail/*" --chmod=D0770,F0640 --groupmap=*:SEC_lab_VIR   grid@10.2.31.23:/data/COVID19/ARTIC/${RUN_ID} /NGS/active/VIR/SARS-CoV2/run_links/MASC/
    
    bash NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_artic_96_run_V4.sh -c ./Artic_pipeline_config -l M $RUN_ID

### KSC gridIon artic run
     bash NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_artic_96_run.sh -c ./Artic_pipeline_config -l K $RUN_ID

### KSC gridIon rapid run
    NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_rapid.sh -c ./Artic_pipeline_config -l K -p V1200 $RUN_ID

### CSC gridIon artic run

    rsync -zavm --include="*/" --exclude '*.fast5' --exclude '*/fast5_fail/*' --exclude '*/fastq_fail/*' --perms --group --chmod=D0770,F0640 --groupmap=*:SEC_lab_VIR  grid@10.3.30.100:/data/GridION_CSC/COVID19/ARTIC/${RUN_ID} /NGS/active/VIR/SARS-CoV2/run_links/CSC

#### **assembly** (Una: for now. I didn't want to accidentaly break other stuff.)

    bash NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_artic_96_run_csc.sh -c ./Artic_pipeline_config -l C $RUN_ID

### CSC gridIon rapid run

    rsync -zavm --include="*/" --exclude '*.fast5' --exclude '*/fast5_fail/*' --exclude '*/fastq_fail/*' --perms --group --chmod=D0770,F0640 --groupmap=*:SEC_lab_VIR  grid@10.3.30.100:/data/GridION_CSC/COVID19/RAPID/${RUN_ID} /NGS/active/VIR/SARS-CoV2/run_links/CSC
    
    NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_rapid.sh -c ./Artic_pipeline_config -l C -p V1200 $RUN_ID


### CSC MinION artic run -- deprecated
**./NZ_SARS-CoV-2_genomics/Artic_pipeline/minion_artic_run.sh -c ./Artic_pipeline_config -l C $RUN_ID -p V3**

**Note: make sure the correct primer is used. It can be V3, V1200, V2500. V3 is Artic, V1200 is Rapid. The default is V1200.**

### general instructions
These are the scripts that are run by the user. They find the raw data, set up the analysis directories, then they call some sbatch scripts to analyses the data. The scripts are all very similar. The script to call depends on what kind of sequencing library was run, on what type of machine (minion or gridion). The sections below try to explain what one to use. 
The run scripts use a config file to store paths to data and environments ect so these are not exposed to public if we publish the scripts.


The general way to call any run script is:

`./NZ_SARS-CoV-2_genomics/Artic_pipeline/run_script.sh -c ./Artic_config_file -l <str> $RUN_ID`

NZ_SARS-CoV-2_genomics/Artic_pipeline/minion_artic_run.sh -c ./Artic_pipeline_config -l C $RUN_ID  

Where run $RUN_ID is what the run was called on the machine. You would know this from when you rsync'ed the data to production from the sequencer. The `-l` flag is for the location of the run i.e., `K` = KSC, `M` = MASC or `C` = CSC. 

### gridion_artic_96_run.sh
    - This is for the assembly of bash  
    
    #NZ_SARS-CoV-2_genomics/Artic_pipeline/minion_artic_run.sh -c ./Artic_pipeline_config -l C $RUN_ID   (una: This is wrong right?)
    
    eg `./NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_artic_96_run.sh -c ./Artic_pipeline_config -l K $RUN_ID`
    
    ecalled ARTIC-v3 data from the gridIon (at KSC) for libraries that have used the ligation sequencing kit with any barcode kit 1-24 and 1-96.
    - Barcoding/demultiplexing should be turned off on the GridIon as it needs to be done on production for ARTIC protocol to work properly
    - Currently active 
    - I will add extra comments to this script so it is clear what some of the weird Bash stuff does.
    - Default location is KSC so remember to add the `-l M` for runs done in MASC 

### gridion_artic_96_run_V4.sh  

modification of gridion_artic_96_run.sh for V4 runs, tried a couple of times, seems to work. haven't tested for KSC or CSC.  

bash NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_artic_96_run_V4.sh -c ./Artic_pipeline_config -l M $RUN_ID

### gridion_artic_run_V1200_rapid.sh
    - This is for assembly of basecalled and barcoded (demultiplexed) ARTIC-v1200 data produced on the GridIon using the rapid library prep kit. 
    - Basecalling and barcoding must be ON for this to work. 
    - Used for the urgent runs 
    - Currently active
    - This also used the location flag

### minion_artic_run.sh
    - This is for assembly of non-basecalled ARTIC-v3 data produced from the minIon (not gridIon) with the ligation sequencing kit and 1-24 barcode kit. 
    - Uses the GPU server Orac with slurm for basecalling and assumes 1-24 barcoding.
    - This was the first run script, all other run scripts were based on this one. 
    - Not really used anymore since we got a GridIon but could be useful if CSC does COVID sequencing on a minion
    - Could/should upgrade to the 96 barcode kit 

### gridion_artic_run_V1200.sh
    - This is for the assembly of basecalled ARTIC-v1200 data from the gridIon (at KSC) for libraries that have used the ligation sequencing kit/ 1-24 barcode kit.
    - Barcoding/demultiplexing should be turned OFF during the GridIon run as it needs to be done on production for ARTIC protocol.
    - This is rarely used as the v1200 primers set is generally reserved for urgent runs (rapid kit). Sometimes when a genome has a large missing section we may try this protocol 
    - should work but not currently active. Should change the barcodes to 1-96
    - location flag not used in this script will just be default KSC directories.

```bash
bash gridion_artic_96_run.sh -c /NGS/active/VIR/SARS-CoV2/Artic_pipeline_config ${RUN_ID}
```

## Rapid v1200 runs at MASC - Get data


### Find the RUN

Find out what the RUN_ID is if you can. Look on Teams or in `W:\COVID-19\sequencing\MASC\RAPID` if the run sheet have been make and loaded up, otheriwse just look around on their GridIon.

Find where the run is on their machine. They have a data structure of `/data/COVID/RAPID` so it should be somewhere in there:
```bash
ssh grid@10.2.31.23
```
Eg. `grid@GXB03041:/data/COVID/RAPID/20210127_v1200_rapid_MASC`

You may need to search around as this hasn't been very consisitant. You will need to know the full path so note it down if it's not in the right place so you can make changes to the rsync command we will run soon 

export the run_id:
```bash
 export RUN_ID=20210127_v1200_rapid_MASC
```

### rsync the data to the cluster

If the data is in the right place (i.e in `/data/COVID/RAPID`):

```bash
rsync -zavmr --include="*/" --exclude="*.fast5" --exclude="*/fastq_fail/*" --chmod=D0770,F0640 --groupmap=*:SEC_lab_VIR grid@10.2.31.23:/data/COVID19/RAPID/${RUN_ID} /NGS/active/VIR/SARS-CoV2/run_links/MASC/
```

This will put the data in the righ place for the script to find it (i.e `/NGS/active/VIR/SARS-CoV2/run_links/MASC/`). If the data was in a different place on the GridIon you will need to change this `/data/COVID19/RAPID/` part of the command


## Rapid v1200 runs at MASC - Assembly

### Run the assembly script.
 
To keep things tidy, I run these commands from the `SARS-CoV2` directory where this note book is now. If you are not there already, `cd /NGS/active/VIR/SARS-CoV2`


To start the assembly, run this command:

```bash
./NZ_SARS-CoV-2_genomics/Artic_pipeline/gridion_artic_run_V1200_rapid.sh -c ./Artic_pipeline_config -l M $RUN_ID
```

The `-c` flag will point to the required config file. The is one living in `/NGS/active/VIR/SARS-CoV2` where you should be.

The `-l` flag needs to be set to `M` or `MSC` for runs from MSC. It just points to the right data and output locations. 


In [ ]:
## monitor run through the screen or squeue command

# optimisations
- have a per run config file that specifies run_id and design parameter
 - auto-choose script to run based on that config



# dirty 'tricks'
- put all IDs in a file (1 per line)
- adjust head command for number of samples (+1 to get header)

##
while read k; do     grep "$k" /NGS/active/VIR/SARS-CoV2/analysis/BIdownload/BI_epi.tsv | cut -f 2-4,9-11,28,29; done < /NGS/active/VIR/SARS-CoV2/latest.txt | sed 's/\t/,/g' | head -36

##Pangolin 
```bash
singularity exec -B /NGS:/NGS /opt/bioinf/pangolin/2.1.0/pangolin_2.1.0.sif pangolin -t 10 --max-ambig 0.50 --tempdir /NGS/scratch/VIR/SARS-CoV2 /NGS/active/VIR/SARS-CoV2/analysis/GridIon_assemblies/20210106_analysis/lib1/all.fasta
```

In [ ]:
## grep all lineages for latest
while read k; do     grep "$k"  /NGS/active/VIR/SARS-CoV2/analysis/pangolin_lineage/all_lineage.csv; done < /NGS/active/VIR/SARS-CoV2/latest.txt | head -36
